In [1]:
from models.SSD300 import SSD300
import tensorflow as tf
import numpy as np
keras = tf.keras

In [2]:
SSD300_model = SSD300(10)

In [10]:
test_input = tf.ones((20, 300, 300, 3), dtype=tf.dtypes.float32, name=None)
conf, loc = SSD300_model.call(test_input)